In [1]:
from typing import List, Mapping, NamedTuple, Optional

from dotenv import load_dotenv
from langchain import LLMChain
from langchain.embeddings.base import Embeddings
from langchain.chains.router import MultiRouteChain
from langchain.chains.router.base import RouterChain
from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.memory import ZepChatMessageHistory, ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores import DocArrayInMemorySearch

In [2]:
ZEP_API_URL = "http://localhost:8000"

chat_history = ZepChatMessageHistory(session_id="test", url=ZEP_API_URL)
memory = ConversationBufferMemory(chat_history=chat_history)

In [3]:
load_dotenv()

True

In [5]:
sales_template = """You are widgets sales rep and your job is to assist humans with completing the purchase of a widget.
In order to close a widget sale, you need to know how many widgets the human would like to purchase. Don't be pushy about making the sale,
but remember that your job is dependent on achieving your sales quota.

Important sales information:
- The current price of widgets is $499.
- If a customer wants to purchase 10 or more widgets, you can offer a 20% discount. There are no discounts available for smaller purchases.
- They come in blue, black, hot pink, and sparkling gold colors.
- Widgets have a warranty of 1 year.

Here are the prior messages in this conversation:
{chat_history}

Here is a question: {input}
"""

support_template = """You are support agent for a widget producer and your job is to assist humans with issues they may have with using the widgets
they purchased from us.
In order to assist a human, you need to know when the widget was purchased, what color it is, and what the user's support issue is.

Important notes about working with humans:
- Always be friendly! They may be upset if their widget doesn't work. Being friendly will help you getting the answers you need.
- Humans can only answer 1 question at a time.

Important support information:
- Widgets have a warranty of 1 year.
- Sparkling gold widgets tend to flake the sparkling paint. We are happy to replace these as long as the widget is under warranty.
- Many humans forget to power on their widgets before attempting use. This should be your first line of questioning.

Today's date is 06/27/2023.

Here are the prior messages in this conversation:
{chat_history}

Here is a question: {input}
"""

intent_models = [
    IntentModel(
        intent="purchase a widget",
        description="the human would like to make a purchase",
        prompt=sales_template,
        default=True,
    ),
    IntentModel(
        intent="needs customer support",
        description="the human has a support query",
        prompt=support_template,
    ),
]

In [6]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [7]:
class IntentRouterChain(MultiRouteChain):
    """Chain for routing inputs to different chains based on intent."""

    router_chain: RouterChain
    destination_chains: Mapping[str, LLMChain]
    default_chain: LLMChain

    @property
    def output_keys(self) -> List[str]:
        return ["text"]

    @property
    def input_keys(self) -> List[str]:
        return ["input", "chat_history"]

    @classmethod
    def from_intent_models(
        cls,
        intent_models: List[IntentModel],
        llm: ChatOpenAI,
        embedding_model: Optional[Embeddings],
        memory: Optional[ConversationBufferMemory] = None,
        verbose: bool = False,
    ) -> "IntentRouterChain":
        """Create a new IntentRouterChain from a list of intent models."""
        names_and_descriptions = [(i.intent, [i.description]) for i in intent_models]

        router_chain = EmbeddingRouterChain.from_names_and_descriptions(
            names_and_descriptions,
            DocArrayInMemorySearch,
            embedding_model,
            routing_keys=["input"],
            verbose=verbose,
        )

        default_chain: Optional[LLMChain] = None
        destination_chains = {}
        for i in intent_models:
            destination_chains[i.intent] = LLMChain(
                llm=llm,
                prompt=PromptTemplate(
                    template=i.prompt, input_variables=["input", "chat_history"]
                ),
                memory=memory,
            )
            if i.default:
                default_chain = destination_chains[i.intent]

        if not default_chain:
            raise ValueError("No default chain was specified.")

        return cls(
            router_chain=router_chain,
            destination_chains=destination_chains,
            default_chain=default_chain,
            verbose=verbose,
        )

In [8]:
chain = IntentRouterChain.from_intent_models(
    intent_models=intent_models,
    llm=llm,
    embedding_model=OpenAIEmbeddings(),
    verbose=True,
)

In [9]:
print(chain.run("Hello! How much do widgets cost?"))

ValueError: A single string input was passed in, but this chain expects multiple inputs ({'input', 'chat_history'}). When a chain expects multiple inputs, please call it by passing in a dictionary, eg `chain({'foo': 1, 'bar': 2})`

In [16]:
print(chain.run("I'm upset that my widget doesn't work"))



> Entering new  chain...


> Entering new  chain...

> Finished chain.
needs customer support: {'input': "I'm upset that my widget doesn't work"}

> Entering new  chain...
Prompt after formatting:
You are support agent for a widget producer and your job is to assist humans with issues they may have with using the widgets
they purchased from us.
In order to assist a human, you need to know when the widget was purchased, what color it is, and what the user's support issue is.

Import notes about working with humans:
- Always be friendly! They may be upset if their widget doesn't work. Being friendly will help you getting the answers you need.
- Humans can only answer 1 question at a time.

Important support information:
- Widgets have a warranty of 1 year.
- Sparkling gold widgets tend to flake the sparkling paint. We are happy to replace these as long as the widget is under warranty.
- Many humans forget to power on their widgets before attempting use. This should be your first line o